# Libraries

In [1]:
import csv
import glob
import uuid
import json
from infostop import Infostop
from pymongo import MongoClient
import pandas as pd
import h3
#import googlemaps
from geopy.geocoders import Nominatim

import geopandas as gpd
import shapely
from multiprocessing import Pool
import time
import os
import random
import logging

In [2]:
geolocator = Nominatim(user_agent="myapp")
coordinates = "{}, {}".format(-11.984677099999999, -77.09265620000002)

info = geolocator.reverse(coordinates)

info.raw

{'place_id': 117738090,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 121371938,
 'lat': '-11.984585359454531',
 'lon': '-77.0926334681423',
 'display_name': 'Los Pinos Del Norte, Urb. Virgen del Rosario ll Etapa, San Martín de Porres, Lima, 15109, Perú',
 'address': {'road': 'Los Pinos Del Norte',
  'neighbourhood': 'Urb. Virgen del Rosario ll Etapa',
  'suburb': 'San Martín de Porres',
  'city': 'San Martín de Porres',
  'region': 'Lima',
  'state': 'Lima',
  'postcode': '15109',
  'country': 'Perú',
  'country_code': 'pe'},
 'boundingbox': ['-11.9847309', '-11.9843834', '-77.0932621', '-77.0920461']}

In [30]:
info.raw['address']['city']

'Comas'

# Save to mongodb

- H3: https://towardsdatascience.com/uber-h3-for-data-analysis-with-python-1e54acdcc908
- openstreet normatim https://nominatim.org/release-docs/develop/api/Reverse/

In [2]:
# client = MongoClient("mongodb://mongo.as-dell.copernic.local:27016")
# db = client["covid19"]
# db
# importing pymongo
from pymongo import MongoClient

# establing connection
try:
#     connect = MongoClient()
#     connect = MongoClient("mongodb://mongo.as-dell.copernic.local:27016")
    connect = MongoClient("mongodb://172.17.4.235:27017")
    print("Connected successfully!!!")
except:
    print("Could not connect to MongoDB")

# connecting or switching to the database
db = connect.covid19


Connected successfully!!!


In [3]:
cursor = db.user_locations.find()
for record in cursor:
    print(record)

ServerSelectionTimeoutError: 172.17.4.235:27017: [Errno 113] No route to host

In [3]:
# file = ""
jsonFiles = glob.glob("data/devices_json/*.json")
jsonFiles

['data/devices_json/FILE7424afef-22c4-42c8-8a6c-9fa537499329.json',
 'data/devices_json/FILEa7e94e8b-c6b9-4cb2-a5ff-a509f1205d4d.json',
 'data/devices_json/FILE7709cff9-43b8-4568-ba0d-e61ca70a94d0.json',
 'data/devices_json/FILE3a6714f7-9b15-477f-b04b-b3d9fed26fe9.json',
 'data/devices_json/FILE8036789b-87a9-4b6f-be1b-e16880f02f1a.json',
 'data/devices_json/FILE640c4604-6d2e-434c-8888-39530ddd72d5.json',
 'data/devices_json/FILEf8adf412-f639-45ff-b15c-d40c136c9a61.json',
 'data/devices_json/FILE61559306-5160-4cbd-8c62-baa7830a1538.json',
 'data/devices_json/FILE67ab34c6-6b2f-4944-9d1a-b96005766193.json',
 'data/devices_json/FILE06c6e0a2-bc72-4cc5-ac7e-d9ca2049244d.json',
 'data/devices_json/FILEdbe9005b-dd8c-4ad4-b397-e042a53bfb14.json',
 'data/devices_json/FILEacd84e34-68ba-406d-8358-99381909fe77.json',
 'data/devices_json/FILEe159de13-c0b0-4b92-96b3-88904f063f2d.json',
 'data/devices_json/FILEeec45510-3dbc-4053-bbfd-dd2f7f509df3.json',
 'data/devices_json/FILEbbaaa1bb-e1d8-4bb8-b91c-

In [4]:
len(jsonFiles)
file = jsonFiles[0]
with open(file , 'r') as fp:
    record = fp.read()
    record = json.loads(record)
print(type(record))
record


<class 'dict'>


{'_id': '7424afef-22c4-42c8-8a6c-9fa537499329',
 'location': [{'index': 0,
   'date': '2020-04-06 00:05:46',
   'lat': -11.9517009,
   'lon': -77.0941776,
   'label': -1,
   'h3_10': '8a8e62d1b79ffff',
   'h3_9': '898e62d1b7bffff',
   'h3_8': '888e62d1b7fffff',
   'h3_7': '878e62d1bffffff',
   'h3_6': '868e62c37ffffff'},
  {'index': 1,
   'date': '2020-05-07 22:03:16',
   'lat': -11.9527439,
   'lon': -77.09350129999999,
   'label': -1,
   'h3_10': '8a8e62d1b6a7fff',
   'h3_9': '898e62d1b6bffff',
   'h3_8': '888e62d1b7fffff',
   'h3_7': '878e62d1bffffff',
   'h3_6': '868e62c37ffffff'},
  {'index': 2,
   'date': '2020-05-07 22:04:12',
   'lat': -11.9528036,
   'lon': -77.093583,
   'label': -1,
   'h3_10': '8a8e62d1b6a7fff',
   'h3_9': '898e62d1b6bffff',
   'h3_8': '888e62d1b7fffff',
   'h3_7': '878e62d1bffffff',
   'h3_6': '868e62c37ffffff'},
  {'index': 3,
   'date': '2020-05-07 22:06:27',
   'lat': -11.952218499999999,
   'lon': -77.0939985,
   'label': -1,
   'h3_10': '8a8e62d1b79ff

In [12]:
record['location'][0]

{'index': 0,
 'date': '2020-04-06 00:05:46',
 'lat': -11.9517009,
 'lon': -77.0941776,
 'label': -1,
 'h3_10': '8a8e62d1b79ffff',
 'h3_9': '898e62d1b7bffff',
 'h3_8': '888e62d1b7fffff',
 'h3_7': '878e62d1bffffff',
 'h3_6': '868e62c37ffffff'}

In [283]:

# creating or switching to demoCollection
collection = db.user_locations

# first document
document1 = {
        "name":"John",
        "age":24,
        "location":"New York"
        }
#second document
document2 = {
        "name":"Sam",
        "age":21,
        "location":"Chicago"
        }

document1 = record

# Inserting both document one by one
collection.insert_one(document1)
# collection.insert_one(document2)



# Printing the data inserted
cursor = collection.find()
for record in cursor:
    print(record)

Connected successfully!!!


ServerSelectionTimeoutError: mongo.as-dell.copernic.local:27016: [Errno -3] Temporary failure in name resolution

# Logger

In [257]:
logger = logging.getLogger('mongodb_log')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('mongodb.log', mode='w')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
formatter = logging.Formatter('%(asctime)s | %(name)s | %(levelname)s | %(process)d | %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

# Save to Json 

In [264]:
def parse_traj(file):
    model = Infostop()

    df = pd.read_csv(file, sep=',')
    df.columns = ["user_id", "date", "lon", "lat"]
    df["date"] = pd.to_datetime(df.date)
    df.sort_values(by="date", inplace=True)
    df["ts"] = (df["date"] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

    traj = df[["lat", "lon", "ts"]].values
    model = Infostop(r1=50, r2=50, min_staying_time=200)
    labels = model.fit_predict(traj)
    df["label"] = labels
    logger.info("Parse trajectorie OK | {}".format(file))
    return df

def create_record(df, use_geocoding=False):
    geolocator = Nominatim(user_agent="myapp")

    record = {}
    record["_id"] = df["user_id"].iloc[0]
    location = []
    idx = 0
    prev_coordinates = ""
    for k,row in df.iterrows():
        
        loc = {
                "index": idx, 
                "date": str(row["date"]), 
                "lat": row["lat"], 
                "lon": row["lon"], 
                "label": row["label"],
                "h3_10" : h3.geo_to_h3(row["lat"], row["lon"], 10),
                "h3_9" : h3.geo_to_h3(row["lat"], row["lon"], 9),
                "h3_8" : h3.geo_to_h3(row["lat"], row["lon"], 8),
                "h3_7" : h3.geo_to_h3(row["lat"], row["lon"], 7),
                "h3_6" : h3.geo_to_h3(row["lat"], row["lon"], 6)
        }
        
        if use_geocoding:
            coordinates = "{}, {}".format(row["lat"], row["lon"])
            if prev_coordinates == coordinates:
                info = prev_info
            else:
                info = geolocator.reverse(coordinates)
            
            if "road" in info.raw["address"].keys(): 
                loc["road"] = info.raw["address"]["road"]
            if "city" in info.raw["address"].keys():
                loc["city"] = info.raw["address"]["city"]
            if "region" in info.raw["address"].keys():
                loc["region"] = info.raw["address"]["region"]
            if "shop" in info.raw["address"].keys():
                loc["shop"] = info.raw["address"]["shop"]
                
            prev_coordinates = coordinates
            prev_info = info
       
        location.append(loc)
        idx += 1
    record["location"] = location
    return record

def preProcessFile(file):
    tmp = pd.read_csv(file).sort_values('datetime').dropna()
    cond = (abs(tmp.x)<=180) & (abs(tmp.y)<=90)
    tmp[cond].to_csv(file, index=False)

def processJson (file, saveFile=True):
    filesPath = 'Data/devices_list' #path for csv files
    jsonPath = 'Data/devices_json' #path for json files
    filepath = os.path.join(filesPath, file)
    try:
#         preProcessFile(filepath) #pre process to quit Nan records and validate coordinates
        df = parse_traj(filepath)
        record = create_record(df, use_geocoding=False)
        if saveFile:
            fileJson = file.replace('.csv', '.json')
            fileJsonPath = os.path.join(jsonPath, fileJson )
            with open(fileJsonPath, 'w') as fp:
                json.dump(record, fp, indent=3)
                logger.info("File OK | {}".format(file))
                return fileJsonPath
        return record
    except Exception as e:
        logger.error("Error in file: | {} | {}".format(file, e))

In [261]:
jsons = os.listdir("Data/devices_json")
jsons = [x.replace('.json', '.csv') for x in jsons]
# jsons[:10]

files = os.listdir("Data/devices_list")

toProcess = list(set(files) - set(jsons))
print(len(toProcess))
toProcess[:10]

In [260]:
##Main execution using multiprocessing
if __name__ == '__main__':
    ts = time.time()
    pool = Pool(os.cpu_count())                         # Create a multiprocessing Pool

    files = toProcess

    logger.info("Total files: {}".format(len(files)))
    
    pool.map(processJson, files)
    logger.info('Process finished in {} seconds'.format(time.time() - ts))

# Others test

In [254]:
#Parse Trajectorie
file = 'Data/devices_list/FILEe98001c8-56a0-4b71-9aa3-196132244373.csv'
# df = parse_traj(tmp)

model = Infostop()

df = pd.read_csv(file, sep=',')
# df.columns = ["user_id", "date", "lon", "lat"]
# df["date"] = pd.to_datetime(df.date)
# df.sort_values(by="date", inplace=True)
# df["ts"] = (df["date"] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

# traj = df[["lat", "lon", "ts"]].values
# model = Infostop(r1=50, r2=50, min_staying_time=200)
# # model = Infostop()
# labels = model.fit_predict(traj)
# df["label"] = labels
# logger.info("Parse trajectorie OK | {}".format(file))
df.sort_values('x')

,device_id,datetime,x,y
643,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-07 18:44:22,-73.266066,-3.733976
642,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-07 18:40:22,-73.263083,-3.734711
644,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-07 18:56:28,-73.263083,-3.734711
702,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-10 22:16:35,-73.262360,-3.729108
297,e98001c8-56a0-4b71-9aa3-196132244373,2020-05-26 11:57:01,-73.262082,-3.723322
...,...,...,...,...
684,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-10 16:01:34,-73.241429,-3.722634
751,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-11 18:47:25,-73.241107,-3.722492
938,e98001c8-56a0-4b71-9aa3-196132244373,2020-10-15 02:55:42,-73.240980,-3.738245
698,e98001c8-56a0-4b71-9aa3-196132244373,2020-08-10 18:36:16,-73.240847,-3.727654


In [253]:
file = 'Data/devices_list/FILEe98001c8-56a0-4b71-9aa3-196132244373.csv'
preProcessFile(file)

In [246]:
cond = (abs(df.x)<=180) & (abs(df.y)<=90)
df[cond]
# df[abs(df.y)<=90]

,device_id,datetime,x,y
0,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-04-08 00:12:14,-76.962134,-12.171806
1,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-04-08 00:17:42,-76.957517,-12.138230
2,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-04-08 03:33:16,-76.962135,-12.171798
3,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-04-08 03:43:40,-76.957324,-12.141079
4,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-04-08 18:26:22,-76.957718,-12.140648
...,...,...,...,...
27469,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-10-02 19:03:15,-76.958311,-12.062692
27470,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-10-02 19:09:18,-76.956924,-12.064008
27471,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-10-02 19:09:19,-76.956924,-12.064008
27472,73d8e0bb-a334-4b55-bc34-6b79666f1260,2020-10-02 19:12:08,-76.958365,-12.061629


In [216]:
errorsfiles = [' FILE6c73d12e-5958-4b96-9552-61d35765a23a.csv ',
       ' FILEcc7fa9ef-c489-4fe1-ab20-844359d9e3c6.csv ',
       ' FILEb6afc9e5-a3c0-476e-8bf0-ba359cd8ed36.csv ',
       ' FILE370f3559-cb93-40ae-87a7-e5599aa65fd4.csv ',
       ' FILE4877d4b5-ae59-4352-910e-d7bc1519443e.csv ',
       ' FILE62441a84-ea6a-4e84-822a-9a5e3fae90ba.csv ',
       ' FILE6ffdc13c-0ec9-4990-82f8-bfa696854d41.csv ',
       ' FILE7c52b9a5-ed9d-4491-ba51-8f44fb80c215.csv ',
       ' FILEe2720ae2-32bc-4696-9a49-27ff47576416.csv ',
       ' FILE76aad063-3183-45f8-a97d-0a27daebc0ce.csv ',
       ' FILEaebefa26-0fc3-424c-b5d4-49b925368368.csv ',
       ' FILEa461c137-d038-4b9e-a774-8e0968286aea.csv ',
       ' FILEbc2b829b-f867-41bc-8263-1f0d2af9ff42.csv ',
       ' FILE335602ad-0464-482a-a4fc-6a2146897efe.csv ',
       ' FILEb9a50951-88cf-44c9-9fc2-1c6f11b55b3e.csv ',
       ' FILEe10399a2-9d6c-4507-96a4-dde3cbbd5a14.csv ',
       ' FILEa54ddd5a-434a-4f0b-ba5a-c03a9e79da18.csv ',
       ' FILEb9cefdf1-09da-49b0-9d60-31af108fe058.csv ',
       ' FILEcec338de-9407-4a62-9cff-5e634321bcb3.csv ',
       ' FILE60127132-9142-4323-996c-5fbc850eea39.csv ',
       ' FILE90e06920-7912-4675-85f2-8eb0d1c392ac.csv ',
       ' FILE7ef45248-05a7-4e88-a8c0-8b1e40fe8ee1.csv ',
       ' FILEb4452e24-be2c-4eaa-bab3-52bc650d6839.csv ',
       ' FILE8c7adf93-a841-473c-a9d3-f2054f2420c5.csv ',
       ' FILE63057238-d3fd-4156-ba96-194aa01c8e9e.csv ',
       ' FILEef98d2cd-164c-4865-8b68-08a40f0a3a72.csv ',
       ' FILE61b4e282-d7d2-4fb8-a742-87929c45dc70.csv ',
       ' FILE2c94df0b-6c97-4f3a-9b11-ebf83d66f001.csv ',
       ' FILEcbb9a076-58df-4ff3-8dff-56789dbfaa00.csv ']
errorsfiles = [x.strip() for x in errorsfiles]
errorsfiles[:10]

['FILE6c73d12e-5958-4b96-9552-61d35765a23a.csv',
 'FILEcc7fa9ef-c489-4fe1-ab20-844359d9e3c6.csv',
 'FILEb6afc9e5-a3c0-476e-8bf0-ba359cd8ed36.csv',
 'FILE370f3559-cb93-40ae-87a7-e5599aa65fd4.csv',
 'FILE4877d4b5-ae59-4352-910e-d7bc1519443e.csv',
 'FILE62441a84-ea6a-4e84-822a-9a5e3fae90ba.csv',
 'FILE6ffdc13c-0ec9-4990-82f8-bfa696854d41.csv',
 'FILE7c52b9a5-ed9d-4491-ba51-8f44fb80c215.csv',
 'FILEe2720ae2-32bc-4696-9a49-27ff47576416.csv',
 'FILE76aad063-3183-45f8-a97d-0a27daebc0ce.csv']

In [217]:

for file in errorsfiles:
    array = []
    file = 'Data/devices_list/{}'.format(file)
    with open(file, 'r') as fp:
    #     print(fp.read())
    #     cad = fp.read()
        for line in fp:
            array.append(line.replace('\n',''))

    dftmp = pd.DataFrame([x.split(',') for x in array if len(x.split(',')) <= 4][1:] ,
                 columns=['device_id', 'datetime', 'x', 'y'],
    #              dtype=('str', 'datetime', 'float','float')
                )
    dftmp = dftmp.astype({'x':float, 'y':float})
    dftmp.to_csv(file, index=False)

In [212]:
dftmp.dtypes

device_id     object
datetime      object
x            float64
y            float64
dtype: object

In [268]:
tmp = 'Data/devices_json/FILEa352077c-87c4-46db-8a78-2c4441204d6f.json'
with open(tmp, 'r') as file:
    print(file.read())


{
   "_id": "a352077c-87c4-46db-8a78-2c4441204d6f",
   "location": [
      {
         "index": 0,
         "date": "2020-04-07 03:17:18",
         "lat": -11.8527746,
         "lon": -77.0644351,
         "label": 1,
         "h3_10": "8a8e62d00d17fff",
         "h3_9": "898e62d00d3ffff",
         "h3_8": "888e62d00dfffff",
         "h3_7": "878e62d00ffffff",
         "h3_6": "868e62d07ffffff"
      },
      {
         "index": 1,
         "date": "2020-04-07 03:19:02",
         "lat": -11.8528214,
         "lon": -77.0644777,
         "label": 1,
         "h3_10": "8a8e62d00d17fff",
         "h3_9": "898e62d00dbffff",
         "h3_8": "888e62d00dfffff",
         "h3_7": "878e62d00ffffff",
         "h3_6": "868e62d07ffffff"
      },
      {
         "index": 2,
         "date": "2020-04-07 03:27:57",
         "lat": -11.8528085,
         "lon": -77.06446770000002,
         "label": 1,
         "h3_10": "8a8e62d00d17fff",
         "h3_9": "898e62d00dbffff",
         "h3_8": "888e62d00df

In [108]:
# pd.read_csv('Data/devices_list/FILEc1cde892-8560-402d-8946-edb28aff5dec.csv')
# pd.read_csv('Data/devices_list/FILEc1cde892-8560-402d-8946-edb28aff5dec.csv')
tmp = 'Data/devices_list/FILEa352077c-87c4-46db-8a78-2c4441204d6f.csv'
# parse_traj(tmp)
x = pd.read_csv(tmp).sort_values('datetime').dropna()
x.to_csv(tmp, index=False)
# x
# .to_csv(tmp, index=False)
#.dropna()

In [109]:
pd.read_csv(tmp)

,device_id,datetime,x,y
0,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-07 03:17:18,-77.064435,-11.852775
1,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-07 03:19:02,-77.064478,-11.852821
2,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-07 03:27:57,-77.064468,-11.852809
3,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-07 03:29:10,-77.064474,-11.852791
4,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-07 03:30:03,-77.064471,-11.852828
...,...,...,...,...
204,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-20 17:44:47,-77.064442,-11.852759
205,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-20 17:47:55,-77.064442,-11.852758
206,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-20 17:50:27,-77.064438,-11.852668
207,a352077c-87c4-46db-8a78-2c4441204d6f,2020-04-20 17:55:24,-77.064438,-11.852756


In [34]:
files[481698]

'FILE50da1735-1871-4001-80ef-afd930852454.csv'

In [31]:
%%time
rec = []
for file in files[:10]:
    df = parse_traj(file)
    # geoconding could be time consumming
    record = create_record(df, use_geocoding=False)
    #db.user_locations.insert_one(record)
    rec.append(record)

CPU times: user 9.28 s, sys: 403 ms, total: 9.69 s
Wall time: 11.7 s


# Export to geojson

In [25]:
def h3_to_geojson_feature(uber_h3, level=0):
    import geojson
    import h3
    
    poly = geojson.Polygon(
        h3.h3_to_geo_boundary(uber_h3, geo_json=True)
    )
    return geojson.Feature(geometry=poly, properties={'h3': uber_h3, 'level':level})

def h3_to_shapely(uber_h3):
    from shapely.geometry import Polygon
    return Polygon(h3.h3_to_geo_boundary(uber_h3, geo_json=True))
    
def features_to_file(features, filename):
    from geojson import FeatureCollection, dump
    
    feature_collection = FeatureCollection(features)
    with open(filename, 'w') as f:
        dump(feature_collection, f)

In [ ]:
features = []
for loc in rec[0]["location"]:
    features.append(h3_to_geojson_feature(loc["h3_10"], level=10))
features_to_file(features, "test_h10.json")

features = []
for loc in rec[0]["location"]:
    features.append(h3_to_geojson_feature(loc["h3_8"], level=8))
features_to_file(features, "test_h8.json")

features = []
for loc in rec[0]["location"]:
    features.append(h3_to_geojson_feature(loc["h3_9"], level=9))
features_to_file(features, "test_h9.json")

In [32]:
%%time
import os
# folder = "Turismo/bd/"
folder = "Data/devices_json/"
size = 16*1024**2
files = [(os.path.join(folder,file),os.path.getsize(os.path.join(folder,file))) for file in os.listdir(folder) if os.path.getsize(os.path.join(folder,file)) > size]
files[:10]

CPU times: user 3.69 s, sys: 15.4 s, total: 19.1 s
Wall time: 36.8 s


[('Data/devices_json/FILE1f9e342f-4ccc-456f-8a04-3501922d224a.json', 19520927),
 ('Data/devices_json/FILEd4054349-0fff-45be-9a97-f1132fc74490.json', 17125477),
 ('Data/devices_json/FILE80c3c451-c976-4426-b0f5-1e961735d11c.json', 25237351),
 ('Data/devices_json/FILE34d3fbe0-10ee-4950-8929-05ec44cc6088.json', 18667533),
 ('Data/devices_json/FILE2ba86040-9574-4cf8-90a4-243b6b5efaea.json', 24149755),
 ('Data/devices_json/FILE4249f803-0889-43ab-bd51-fc7a0b934582.json', 17193533),
 ('Data/devices_json/FILE95d4fdd2-8939-4524-bcef-7b29f1fcfc34.json', 20054323),
 ('Data/devices_json/FILE9f182af3-cd9b-4a71-af5f-ea29b6ad3b90.json', 28925158),
 ('Data/devices_json/FILEffc3a44a-49f7-4fc4-9afc-7c0a32a8e111.json', 24148954),
 ('Data/devices_json/FILE9f526801-2b54-44ac-a853-a22b46fec3aa.json', 21199694)]

In [51]:
import pandas as pd
tmp = [(x[0].split('/')[2], x[1]/1024**2)  for x in files]
pd.DataFrame(tmp, columns=['File', 'Mb']).to_csv("Data/devices_gt16mb.txt", index=False)
# pd.DataFrame() #.to_csv("Data/devices_gt16mb.txt", index=False)

In [56]:
import numpy as np
np.mean([x[1] for x in tmp])

21.02705539406548

In [25]:
[(file, os.path.getsize(os.path.join(folder,file))/1024**2) for file in os.listdir(folder)]

[('MatrixTourists2016.csv', 272.23051357269287),
 ('BankTransactions2016.csv', 7389.918635368347),
 ('datos_no_outliers.csv', 41.44740295410156),
 ('trx201611.csv', 1589.9901456832886),
 ('mcc.csv', 0.07735157012939453),
 ('trx201610.csv', 1639.9622783660889),
 ('MatrixTouristsCoicop.csv', 273.20396423339844),
 ('trx201609.csv', 1368.9811391830444),
 ('trx2016_tourists_summary.csv', 147.7527618408203),
 ('trx201607.csv', 1721.8530340194702),
 ('trxdata_2016.csv', 256.84004974365234),
 ('merchant.csv', 526.8002033233643),
 ('trx201612.csv', 1984.8853187561035),
 ('trx201608.csv', 1579.082571029663),
 ('Bank2016.csv', 4627.201347351074),
 ('trx2016_summary.csv', 0.008769035339355469),
 ('trx201606.csv', 1425.1477155685425),
 ('MatrixTourists.csv', 117.2811222076416),
 ('BankTransactions2016_small.csv', 4369.166612625122),
 ('trx2016_tourists.csv', 2965.462881088257),
 ('trx2016.csv', 3701.4315576553345),
 ('BankClients.csv', 46.639657974243164)]

In [24]:
16*1024

16384